In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2025-04-28 14:04:04.440678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745863444.473022    7685 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745863444.483419    7685 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745863444.505868    7685 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745863444.505918    7685 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745863444.505920    7685 computation_placer.cc:177] computation placer alr

In [2]:
# Check device that tensorflow is using
physical_devices = tf.config.list_physical_devices()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))


Num GPUs Available:  1
Num CPUs Available:  1


In [3]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

# Check the data shape
print("Ratings shape: ", ratings.cardinality().numpy())
print("Movies shape: ", movies.cardinality().numpy())


I0000 00:00:1745863451.092728    7685 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6073 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 Super with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


Ratings shape:  100000
Movies shape:  1682


In [4]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': np.float32(45.0),
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': np.float32(46.0),
 'timestamp': np.int64(879024327),
 'user_gender': np.True_,
 'user_id': b'138',
 'user_occupation_label': np.int64(4),
 'user_occupation_text': b'doctor',
 'user_rating': np.float32(4.0),
 'user_zip_code': b'53211'}


2025-04-28 14:04:11.747347: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-04-28 14:04:11.773744: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-04-28 14:04:11.784514: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


2025-04-28 14:04:11.902163: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-04-28 14:04:11.913916: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)


In [8]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

2025-04-28 14:04:21.128149: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

In [9]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [10]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [11]:
# Ensure that the movie titles are properly encoded as strings
candidate_embeddings = movies.batch(128).map(lambda x: movie_model(tf.convert_to_tensor(x, dtype=tf.string)))

metrics = tfrs.metrics.FactorizedTopK(
  candidates=candidate_embeddings
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [12]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [13]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [14]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [15]:
model.fit(cached_train, epochs=3)

Epoch 1/3


I0000 00:00:1745863468.967881    7831 service.cc:152] XLA service 0x7f786fb56290 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745863468.967951    7831 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 Super with Max-Q Design, Compute Capability 7.5
2025-04-28 14:04:28.994467: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745863469.068083    7831 cuda_dnn.cc:529] Loaded cuDNN version 90701
I0000 00:00:1745863469.443837    7831 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 [==============================] - 26s 2s/step - factorized_top_k/top_1_categorical_accuracy: 3.3750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0039 - factorized_top_k/top_10_categorical_accuracy: 0.0097 - factorized_top_k/top_50_categorical_accuracy: 0.0667 - factorized_top_k/top_100_categorical_accuracy: 0.1356 - loss: 69838.2138 - regularization_loss: 0.0000e+00 - total_loss: 69838.2138
Epoch 2/3
10/10 [==============================] - 16s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0011 - factorized_top_k/top_5_categorical_accuracy: 0.0125 - factorized_top_k/top_10_categorical_accuracy: 0.0278 - factorized_top_k/top_50_categorical_accuracy: 0.1399 - factorized_top_k/top_100_categorical_accuracy: 0.2612 - loss: 67496.5838 - regularization_loss: 0.0000e+00 - total_loss: 67496.5838
Epoch 3/3
10/10 [==============================] - 18s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0014 - factorized_top_k/top_5_categorical_accuracy: 0.0191 - fac

In [16]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 10s 860ms/step - factorized_top_k/top_1_categorical_accuracy: 8.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0078 - factorized_top_k/top_10_categorical_accuracy: 0.0194 - factorized_top_k/top_50_categorical_accuracy: 0.1214 - factorized_top_k/top_100_categorical_accuracy: 0.2359 - loss: 31093.0869 - regularization_loss: 0.0000e+00 - total_loss: 31093.0869


{'factorized_top_k/top_1_categorical_accuracy': 0.0008500000112690032,
 'factorized_top_k/top_5_categorical_accuracy': 0.007799999788403511,
 'factorized_top_k/top_10_categorical_accuracy': 0.01940000057220459,
 'factorized_top_k/top_50_categorical_accuracy': 0.12144999951124191,
 'factorized_top_k/top_100_categorical_accuracy': 0.23594999313354492,
 'loss': 28263.037109375,
 'regularization_loss': 0,
 'total_loss': 28263.037109375}

In [17]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)'
 b"Kid in King Arthur's Court, A (1995)" b'Michael (1996)']


In [18]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(index, path)

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["42"])

  print(f"Recommendations: {titles[0][:3]}")

INFO:tensorflow:Assets written to: /tmp/tmpiv40k74j/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpiv40k74j/model/assets


Recommendations: [b'Bridges of Madison County, The (1995)'
 b"Kid in King Arthur's Court, A (1995)" b'Michael (1996)']


In [19]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.user_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

I0000 00:00:1745863533.203344    7685 partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 1682
W0000 00:00:1745863533.213314    7685 gmm_utils.cc:1012] Could not normalize centroid due to zero norm or empty or zero-weight partition.
I0000 00:00:1745863533.223599    7685 kmeans_tree_partitioner_utils.h:90] PartitionerFactory ran in 20.10905ms.


In [20]:
# Get recommendations.
_, titles = scann_index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)'
 b"Kid in King Arthur's Court, A (1995)" b'It Could Happen to You (1994)']


In [21]:
# Export the query model.
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  # Save the index.
  tf.saved_model.save(
      scann_index,
      path,
      options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
  )

  # Load it back; can also be done in TensorFlow Serving.
  loaded = tf.saved_model.load(path)

  # Pass a user id in, get top predicted movie titles back.
  scores, titles = loaded(["42"])

  print(f"Recommendations: {titles[0][:3]}")

INFO:tensorflow:Assets written to: /tmp/tmp6l_9zklg/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp6l_9zklg/model/assets


Recommendations: [b'Bridges of Madison County, The (1995)'
 b"Kid in King Arthur's Court, A (1995)" b'It Could Happen to You (1994)']
